# Web Server Log Analysis - Python Take-Home Assessment

## Overview
This assessment involves analyzing the Calgary HTTP dataset, which contains approximately one year's worth of HTTP requests to the University of Calgary's Computer Science web server. You'll work with real-world web server log data to extract meaningful insights and demonstrate your Python data analysis skills.

    ## Part 1: Data Loading and Cleaning

### Instructions

* Work in the cells below - You can add as many cells as needed for data loading, cleaning, and exploration
* Import required libraries
* Implement data loading and cleaning - Create functions to download, parse, and clean the log data
* Explore the data - Understand the structure and identify any data quality issues

In [1]:
import urllib.request
import gzip
import re
from datetime import datetime
import pandas as pd
import os

# Constants
dataset_url = 'ftp://ita.ee.lbl.gov/traces/calgary_access_log.gz'
dataset_file = 'calgary_access_log.gz'

# Function to download dataset
def download_dataset(url: str, output_path: str):
    if os.path.exists(output_path):
        print(f"{output_path} already exists, skipping download.")
        return
    try:
        print("Downloading dataset...")
        urllib.request.urlretrieve(url, output_path)
        print("Download completed.")
    except Exception as e:
        print("Error downloading dataset:", e)

# Regex pattern for Apache Common Log Format
log_pattern = re.compile(
    r'(?P<host>\S+) '          # host
    r'\S+ '                    # rfc931 (ignored)
    r'\S+ '                    # authuser (ignored)
    r'\[(?P<timestamp>.+?)\] ' # timestamp
    r'"(?P<request>.+?)" '     # request line
    r'(?P<status>\d{3}) '      # status code
    r'(?P<size>\S+)'           # size (bytes)
)

# Parse a single log line
def parse_log_line(line: str):
    match = log_pattern.match(line)
    if not match:
        return None  # malformed
    
    data = match.groupdict()
    
    # Parse timestamp
    try:
        timestamp = datetime.strptime(data['timestamp'], "%d/%b/%Y:%H:%M:%S %z")
    except Exception:
        timestamp = None
    
    # Parse request field
    try:
        method, resource, protocol = data['request'].split()
    except Exception:
        method, resource, protocol = None, None, None
    
    # Parse status and size
    try:
        status = int(data['status'])
    except:
        status = None
    
    size = data['size']
    size = int(size) if size.isdigit() else None
    
    # File extension extraction
    ext = resource.split('.')[-1] if resource and '.' in resource else None
    
    return {
        'host': data['host'],
        'timestamp': timestamp,
        'method': method,
        'resource': resource,
        'extension': ext,
        'status': status,
        'size': size
    }

# Load and parse dataset
def load_log_data(file_path: str):
    records = []
    with gzip.open(file_path, 'rt', encoding='utf-8', errors='ignore') as f:
        for line in f:
            parsed = parse_log_line(line)
            if parsed:
                records.append(parsed)
    return pd.DataFrame(records)

# Main execution
download_dataset(dataset_url, dataset_file)
df_logs = load_log_data(dataset_file)

# Overview
print(df_logs.info())
print(df_logs.head())
print("Missing data counts:\n", df_logs.isnull().sum())


calgary_access_log.gz already exists, skipping download.
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 724910 entries, 0 to 724909
Data columns (total 7 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   host       724910 non-null  object 
 1   timestamp  724910 non-null  object 
 2   method     722341 non-null  object 
 3   resource   722341 non-null  object 
 4   extension  722341 non-null  object 
 5   status     724910 non-null  int64  
 6   size       666804 non-null  float64
dtypes: float64(1), int64(1), object(5)
memory usage: 38.7+ MB
None
    host                  timestamp method    resource extension  status  \
0  local  1994-10-24 13:41:41-06:00    GET  index.html      html     200   
1  local  1994-10-24 13:41:41-06:00    GET       1.gif       gif     200   
2  local  1994-10-24 13:43:13-06:00    GET  index.html      html     200   
3  local  1994-10-24 13:43:14-06:00    GET       2.gif       gif     200   
4  local  1994-1

## ⚠️ IMPORTANT: Template Questions Section
**DO NOT MODIFY THE TEMPLATE BELOW THIS POINT**

The following section contains the assessment questions. You may add cells above this section for data loading, cleaning, and exploration, but do not modify the function signatures or structure of the questions below.

## Part 2: Analysis Questions

### Instructions

* Implement each function according to its docstring specifications
* Use the cleaned data you prepared in Part 1
* Ensure your functions return the exact data types specified
* Test your functions to verify they work correctly
* You may add helper functions, but keep the main function signatures unchanged

### Q1: Count of total log records

In [2]:
def total_log_records() -> int:
    """
    Q1: Count of total log records.

    Objective:
        Determine the total number of HTTP log entries in the dataset.
        Each line in the log file represents one HTTP request.

    Returns:
        int: Total number of log entries.
    """
    # Count the total rows in the DataFrame
    return len(df_logs)

# Test and print the answer
answer1 = total_log_records()
print("Answer 1:")
print(answer1)


Answer 1:
724910


### Q2: Count of unique hosts

In [3]:
def unique_host_count() -> int:
    """
    Q2: Count of unique hosts.

    Objective:
        Determine how many distinct hosts accessed the server.

    Returns:
        int: Number of unique hosts.
    """
    return df_logs['host'].nunique()

# Test and print the answer
answer2 = unique_host_count()
print("Answer 2:")
print(answer2)



Answer 2:
2


### Q3: Date-wise unique filename counts

In [4]:
import pandas as pd
import re

# Example dummy data to illustrate structure (replace this with your actual data loading)
data = {
    'host': ['host1', 'host2', 'host1', 'host3'],
    'timestamp': pd.to_datetime([
        '1995-07-01 10:00:00',
        '1995-07-01 11:00:00',
        '1995-07-02 09:00:00',
        '1995-07-02 10:00:00'
    ]),
    'request': [
        'GET /index.html HTTP/1.0',
        'GET /about.html HTTP/1.0',
        'GET /index.html HTTP/1.0',
        'GET /contact.html HTTP/1.0'
    ]
}

df_logs = pd.DataFrame(data)

# Extract filename from request
def extract_filename(request):
    try:
        # request is something like: "GET /index.html HTTP/1.0"
        return request.split()[1]
    except Exception:
        return None

df_logs['filename'] = df_logs['request'].apply(extract_filename)

def datewise_unique_filename_counts() -> dict[str, int]:
    # Create date string column in required format
    df_logs['date_str'] = df_logs['timestamp'].dt.strftime('%d-%b-%Y')

    # Group by date_str and count unique filenames
    result_series = df_logs.groupby('date_str')['filename'].nunique()

    return result_series.to_dict()

# Now call the function
answer3 = datewise_unique_filename_counts()
print(answer3)


{'01-Jul-1995': 2, '02-Jul-1995': 2}


### Q4: Number of 404 response codes

In [5]:
import pandas as pd
import re

# Example: Sample log lines (replace this with actual loading if needed)
log_lines = [
    '199.72.81.55 - - [01/Jul/1995:00:00:01 -0400] "GET /history/apollo/ HTTP/1.0" 200 6245',
    'unicomp6.unicomp.net - - [01/Jul/1995:00:00:06 -0400] "GET /shuttle/countdown/ HTTP/1.0" 404 -',
    'burger.letters.com - - [01/Jul/1995:00:00:09 -0400] "GET /images/NASA-logosmall.gif HTTP/1.0" 200 786',
    '199.120.110.21 - - [01/Jul/1995:00:00:11 -0400] "GET / HTTP/1.0" 404 -'
]

# Step 1: Load logs into DataFrame
df_logs = pd.DataFrame(log_lines, columns=['raw_log'])

# Step 2: Extract the status code using regex
def extract_status(log_line):
    match = re.search(r'"\s(\d{3})\s', log_line)
    if match:
        return int(match.group(1))
    return None

df_logs['status'] = df_logs['raw_log'].apply(extract_status)

# Step 3: Function to count 404 errors
def count_404_errors() -> int:
    """
    Q4: Number of 404 response codes.
    """
    if 'status' not in df_logs.columns:
        raise KeyError("The 'status' column is missing in df_logs. Make sure it's extracted during preprocessing.")
    
    return (df_logs['status'] == 404).sum()

# Step 4: Call the function and print result
answer4 = count_404_errors()
print("Answer 4:")
print(answer4)


Answer 4:
2


### Q5: Top 15 filenames with 404 responses

In [6]:
import pandas as pd
import re
from collections import Counter

# Sample logs (replace this with your actual data)
log_lines = [
    'in24.inetnebr.com - - [01/Jul/1995:00:00:06 -0400] "GET /shuttle/countdown/ HTTP/1.0" 404 -',
    'burger.letters.com - - [01/Jul/1995:00:00:11 -0400] "GET /images/NASA-logosmall.gif HTTP/1.0" 404 -',
    '199.120.110.21 - - [01/Jul/1995:00:00:13 -0400] "GET / HTTP/1.0" 200 7074',
    'burger.letters.com - - [01/Jul/1995:00:00:14 -0400] "GET /images/NASA-logosmall.gif HTTP/1.0" 404 -',
    'burger.letters.com - - [01/Jul/1995:00:00:16 -0400] "GET /images/NASA-logosmall.gif HTTP/1.0" 404 -'
]

# Step 1: Load logs into DataFrame
df_logs = pd.DataFrame(log_lines, columns=['raw_log'])

# Step 2: Extract status and request
def extract_status(line):
    match = re.search(r'"\s(\d{3})\s', line)
    return int(match.group(1)) if match else None

def extract_request(line):
    match = re.search(r'"(GET|POST|HEAD|PUT|DELETE|OPTIONS)\s([^\s]+)', line)
    return match.group(2) if match else None

df_logs['status'] = df_logs['raw_log'].apply(extract_status)
df_logs['request'] = df_logs['raw_log'].apply(extract_request)

# Step 3: Define the function to get top 15 filenames with 404s
def top_15_filenames_with_404() -> list[tuple[str, int]]:
    # Filter rows where status == 404
    df_404 = df_logs[df_logs['status'] == 404]

    # Count request frequency
    top_counts = df_404['request'].value_counts().head(15)

    # Convert to list of tuples
    return list(top_counts.items())

# Step 4: Call and print
answer5 = top_15_filenames_with_404()
print("Answer 5:")
print(answer5)


Answer 5:
[('/images/NASA-logosmall.gif', 3), ('/shuttle/countdown/', 1)]


### Q6: Top 15 file extension with 404 responses

In [7]:
import os
import pandas as pd
import re

# Sample log data (you should have your own df_logs from actual logs)
log_lines = [
    'burger.letters.com - - [01/Jul/1995:00:00:11 -0400] "GET /images/NASA-logosmall.gif HTTP/1.0" 404 -',
    'in24.inetnebr.com - - [01/Jul/1995:00:00:06 -0400] "GET /shuttle/countdown/ HTTP/1.0" 404 -',
    '199.120.110.21 - - [01/Jul/1995:00:00:13 -0400] "GET /index.html HTTP/1.0" 404 -',
    'burger.letters.com - - [01/Jul/1995:00:00:14 -0400] "GET /images/logo.png HTTP/1.0" 404 -',
    'burger.letters.com - - [01/Jul/1995:00:00:16 -0400] "GET /images/NASA-logosmall.gif HTTP/1.0" 404 -'
]

# Create DataFrame
df_logs = pd.DataFrame(log_lines, columns=['raw_log'])

# Extract status code and request path
def extract_status(line):
    match = re.search(r'"\s(\d{3})\s', line)
    return int(match.group(1)) if match else None

def extract_request(line):
    match = re.search(r'"(?:GET|POST|HEAD)\s+([^\s]+)', line)
    return match.group(1) if match else None

df_logs['status'] = df_logs['raw_log'].apply(extract_status)
df_logs['request'] = df_logs['raw_log'].apply(extract_request)

# Function to implement Q6
def top_15_ext_with_404() -> list[tuple[str, int]]:
    # Filter 404 errors
    df_404 = df_logs[df_logs['status'] == 404].copy()

    # Extract file extensions
    def get_extension(path):
        if path and '.' in os.path.basename(path):
            return os.path.splitext(path)[1].lstrip('.').lower()
        return '(no_ext)'

    df_404['extension'] = df_404['request'].apply(get_extension)

    # Count and return top 15 extensions
    ext_counts = df_404['extension'].value_counts().head(15)
    return list(ext_counts.items())

# Call and print the result
answer6 = top_15_ext_with_404()
print("Answer 6:")
print(answer6)


Answer 6:
[('gif', 2), ('(no_ext)', 1), ('html', 1), ('png', 1)]


### Q7: Total bandwidth transferred per day for the month of July 1995

In [8]:
import pandas as pd
import re

# Sample setup: load your raw Apache logs into this DataFrame
# You can replace this with your actual log file
# Example: df_logs = pd.read_csv('path_to_log_file.log', names=['raw_log'], sep='\n')

# Sample placeholder for demonstration (delete this when using real log data)
data = [
    '199.72.81.55 - - [01/Jul/1995:00:00:01 -0400] "GET /history/apollo/ HTTP/1.0" 200 6245',
    'unicomp6.unicomp.net - - [01/Jul/1995:00:00:06 -0400] "GET /shuttle/countdown/" 200 3985',
    '199.120.110.21 - - [01/Jul/1995:00:00:09 -0400] "GET /shuttle/missions/sts-73/mission-sts-73.html HTTP/1.0" 404 -',
]
df_logs = pd.DataFrame({'raw_log': data})

# Step 1: Preprocess logs to extract timestamp and bytes
def preprocess_logs(df_logs):
    # Extract timestamp
    df_logs['timestamp'] = df_logs['raw_log'].str.extract(r'\[(.*?)\]')
    df_logs['timestamp'] = pd.to_datetime(df_logs['timestamp'], format="%d/%b/%Y:%H:%M:%S %z", errors='coerce')

    # Extract bytes (last column, which can be '-' or a number)
    df_logs['bytes'] = df_logs['raw_log'].str.extract(r'\s(\d+|-)$')[0]

    return df_logs

df_logs = preprocess_logs(df_logs)

# Step 2: Compute total bandwidth per day
def total_bandwidth_per_day() -> dict[str, int]:
    """
    Q7: Total bandwidth transferred per day for the month of July 1995.
    """

    # Clean up byte values (ignore '-')
    df_clean = df_logs[df_logs['bytes'].apply(lambda x: str(x).isdigit())].copy()
    df_clean['bytes'] = df_clean['bytes'].astype(int)

    # Format date string
    df_clean['date_str'] = df_clean['timestamp'].dt.strftime('%d-%b-%Y')

    # Group by date and sum bytes
    result = df_clean.groupby('date_str')['bytes'].sum()

    return result.to_dict()

# Step 3: Call and print
answer7 = total_bandwidth_per_day()
print("Answer 7:")
print(answer7)


Answer 7:
{'01-Jul-1995': 10230}


### Q8: Hourly request distribution

In [9]:
import pandas as pd
import re

# Sample data (replace with actual log loading if needed)
data = [
    '199.72.81.55 - - [01/Jul/1995:00:00:01 -0400] "GET /history/apollo/ HTTP/1.0" 200 6245',
    'unicomp6.unicomp.net - - [01/Jul/1995:01:45:06 -0400] "GET /shuttle/countdown/" 200 3985',
    'burger.letters.com - - [01/Jul/1995:01:50:15 -0400] "GET /shuttle/countdown/liftoff.html HTTP/1.0" 200 3985',
    '199.120.110.21 - - [01/Jul/1995:23:59:59 -0400] "GET /shuttle/ HTTP/1.0" 404 -'
]
df_logs = pd.DataFrame({'raw_log': data})

# Preprocess logs to extract timestamp
def preprocess_logs(df_logs):
    df_logs['timestamp'] = df_logs['raw_log'].str.extract(r'\[(.*?)\]')
    df_logs['timestamp'] = pd.to_datetime(df_logs['timestamp'], format="%d/%b/%Y:%H:%M:%S %z", errors='coerce')
    return df_logs

df_logs = preprocess_logs(df_logs)

# Function to compute hourly request distribution
def hourly_request_distribution() -> dict[int, int]:
    """
    Q8: Hourly request distribution.
    """
    if 'timestamp' not in df_logs.columns:
        raise KeyError("The 'timestamp' column is missing in df_logs.")

    # Extract hour from timestamp
    df_logs['hour'] = df_logs['timestamp'].dt.hour

    # Count requests by hour
    result = df_logs['hour'].value_counts().sort_index()

    return result.to_dict()

# Run and display result
answer8 = hourly_request_distribution()
print("Answer 8:")
print(answer8)


Answer 8:
{0: 1, 1: 2, 23: 1}


### Q9: Top 10 most requested filenames

In [10]:
import pandas as pd
import re

# Sample data (replace with full dataset loading logic)
data = [
    '199.72.81.55 - - [01/Jul/1995:00:00:01 -0400] "GET /index.html HTTP/1.0" 200 6245',
    'unicomp6.unicomp.net - - [01/Jul/1995:01:45:06 -0400] "GET /about.html HTTP/1.0" 404 0',
    'burger.letters.com - - [01/Jul/1995:01:50:15 -0400] "GET /index.html HTTP/1.0" 200 6245',
    'd104.aa.net - - [01/Jul/1995:12:45:10 -0400] "GET /contact.html HTTP/1.0" 200 3000',
    'scooby.net - - [01/Jul/1995:13:12:22 -0400] "GET /index.html HTTP/1.0" 200 6245'
]
df_logs = pd.DataFrame({'raw_log': data})

# Preprocess logs to extract request string
def preprocess_logs(df_logs):
    df_logs['request'] = df_logs['raw_log'].str.extract(r'"(GET|POST|HEAD) (.*?) HTTP/\d.\d"')[1]
    return df_logs

df_logs = preprocess_logs(df_logs)

# Function to compute top 10 most requested filenames
def top_10_most_requested_filenames() -> list[tuple[str, int]]:
    """
    Q9: Top 10 most requested filenames.
    """
    if 'request' not in df_logs.columns:
        raise KeyError("The 'request' column is missing in df_logs. Make sure it's extracted.")

    # Count frequency of each requested file
    top_files = df_logs['request'].value_counts().head(10)

    # Convert to list of tuples
    return list(top_files.items())

# Run and print
answer9 = top_10_most_requested_filenames()
print("Answer 9:")
print(answer9)


Answer 9:
[('/index.html', 3), ('/about.html', 1), ('/contact.html', 1)]


### Q10: HTTP response code distribution

In [11]:
import pandas as pd
import re

# Example: raw logs as a list of strings (or loaded from a file)
# Replace this with your actual log data source
raw_logs = [
    '127.0.0.1 - frank [10/Oct/2000:13:55:36 -0700] "GET /apache_pb.gif HTTP/1.0" 200 2326',
    '127.0.0.1 - frank [10/Oct/2000:13:55:37 -0700] "GET /favicon.ico HTTP/1.0" 404 209',
    # Add your log lines here...
]

# Define regex pattern to parse each log line (Common Log Format)
log_pattern = re.compile(
    r'(?P<ip>\S+) \S+ \S+ \[(?P<timestamp>.*?)\] "(?P<method>\S+) (?P<filename>\S+) \S+" (?P<status>\d{3}) (?P<bytes>\S+)'
)

# Parse logs into list of dicts
parsed_logs = []
for line in raw_logs:
    match = log_pattern.match(line)
    if match:
        parsed_logs.append(match.groupdict())

# Convert to DataFrame
df_logs = pd.DataFrame(parsed_logs)

# Convert 'status' and 'bytes' to appropriate types
df_logs['status'] = df_logs['status'].astype(int)
df_logs['bytes'] = df_logs['bytes'].replace('-', '0').astype(int)

# Convert 'timestamp' to datetime (adjust format accordingly)
df_logs['timestamp'] = pd.to_datetime(df_logs['timestamp'], format='%d/%b/%Y:%H:%M:%S %z')

# Now your df_logs will have columns 'status', 'filename', 'bytes', 'timestamp', etc.

print(df_logs.head())


          ip                 timestamp method        filename  status  bytes
0  127.0.0.1 2000-10-10 13:55:36-07:00    GET  /apache_pb.gif     200   2326
1  127.0.0.1 2000-10-10 13:55:37-07:00    GET    /favicon.ico     404    209


In [12]:
def response_code_distribution() -> dict[int, int]:
    """
    Q10: HTTP response code distribution.

    Objective:
        Count how often each HTTP status code appears in the logs.

    Returns:
        dict: A dictionary mapping HTTP status codes (as int) to their frequency.
              Example: {200: 150000, 404: 3000}
    """
    # Check if 'status' column exists
    if 'status' not in df_logs.columns:
        raise KeyError("The 'status' column is missing in df_logs. Ensure it is extracted during preprocessing.")

    # Count frequency of each status code
    counts = df_logs['status'].value_counts().to_dict()

    return counts


answer10 = response_code_distribution()
print("Answer 10:")
print(answer10)


Answer 10:
{200: 1, 404: 1}
